In [4]:
import polars as pl

In [10]:
df = pl.read_parquet("../data/ledger.parquet")

In [11]:
(
    df
    .select(
        pl.exclude('mcc'),
        mcc=pl.col('mcc').shuffle(),
    )
    .write_parquet("../data/shuffled-mcc-ledger.parquet")
)

In [22]:
from dataclasses import dataclass
from mashumaro.mixins.json import DataClassJSONMixin

@dataclass
class MyOutput(DataClassJSONMixin):
    
    _value: float
    _is_int: bool
    
    @classmethod
    def from_value(cls, value: float|int) -> "MyOutput":
        
        assert isinstance(value, (float, int)), \
            f"value must be of type int or float, not {type(value)}"
        
        if isinstance(value, int):
            return cls(_is_int =True, _value=float(value))
        else:
            return cls(_is_int =False, _value=value)
    
    @property
    def value(self) -> float|int:
        
        if self._is_int:
            return int(self._value)
        else:
            return float(self._value)
        
    def __repr__(self):
        
        _type = "int" if self._is_int else "float"
        return f"{self.__class__.__name__}({self.value}: {_type})"


my_output_instance = MyOutput.from_value(3.1232)
serialized = my_output_instance.to_json()
deserialized = MyOutput.from_json(serialized)

print(deserialized) # MyOutput(3.1232: float)

my_output_instance = MyOutput.from_value(33)
serialized = my_output_instance.to_json()
deserialized = MyOutput.from_json(serialized)

print(deserialized) # MyOutput(33: int)

MyOutput(3.1232: float)
MyOutput(33: int)


In [27]:
from flytekit import workflow, task

@task
def a() -> MyOutput:
    return MyOutput.from_value(32)

@task
def b(arg: MyOutput):
    print(arg)


@workflow
def test():
    result = a()
    b(arg=result)
    
test()

MyOutput(32: int)


In [11]:
deserialized

MyOutput(my_integer_value=1, my_float_value=1.3)